<a href="https://colab.research.google.com/github/marsggbo/AutoMLDemos/blob/master/ch4/nasbenchasr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 下载安装

In [ ]:
!git clone https://github.com/marsggbo/hyperbox.git

In [ ]:
cd hyperbox

In [ ]:
!pip install -e .

# 2. 下载查询数据库

In [ ]:
%%bash
pip install peewee
pip install gdown

if [ -z "${NASBENCHMARK_DIR}" ]; then
    NASBENCHMARK_DIR=~/.hyperbox/nasbenchasr/
fi

echo "Downloading NAS-Bench-ASR..."
wget https://github.com/SamsungLabs/nb-asr/releases/download/v1.1.0/nb-asr-bench-gtx-1080ti-fp32.pickle
wget https://github.com/SamsungLabs/nb-asr/releases/download/v1.1.0/nb-asr-bench-jetson-nano-fp32.pickle
wget https://github.com/SamsungLabs/nb-asr/releases/download/v1.1.0/nb-asr-e10-1234.pickle
wget https://github.com/SamsungLabs/nb-asr/releases/download/v1.1.0/nb-asr-e40-1234.pickle
wget https://github.com/SamsungLabs/nb-asr/releases/download/v1.1.0/nb-asr-e40-1235.pickle
wget https://github.com/SamsungLabs/nb-asr/releases/download/v1.1.0/nb-asr-e5-1234.pickle
wget https://github.com/SamsungLabs/nb-asr/releases/download/v1.1.0/nb-asr-info.pickle

mkdir -p ${NASBENCHMARK_DIR}
mv nb-asr*.pickle ${NASBENCHMARK_DIR}

# 3. 使用NAS-Bench-ASR数据集

## 3.1 导入依赖包

In [2]:
import torch

from hyperbox.mutator import RandomMutator
from hyperbox.networks.nasbenchasr import NASBenchASR

model = NASBenchASR(use_rnn=True)
print(sum([p.numel() for p in model.parameters()]))
rm = RandomMutator(model)

/Users/marsggbo/opt/anaconda3/envs/hyperbox/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


88237349


上面代码中我们构建了 NASBenchASR 超网，其总共的参数量为 84867649

## 3.2 模型编码

`hyperbox`框架基于`https://github.com/SamsungLabs/nb-asr`构建了`NASBenchASR`模型，提供了两种模型编码方式，且两种方式可以互相转换：

- 第一种是以 **列表（list）** 的形式，不过这种方式默认所有单元采用相同的结构，例如，
    - 下面的结构表示单元结构有3个节点，每个节点对应一个list
    - 每个节点的list的第一个元素代表main edge上的操作，其余元素表示skip-connection edges的操作
```python
list_desc = [
    ['linear', 1], # 第一个节点， main edge操作为linear，skip-connection edge的操作是索引值为1的操作，0表示zeroize，1表示identity
    ['conv5', 1, 0], # 第二个节点
    ['conv7d2', 1, 0, 1], # 第三个节点
]
```

- 第二种是以 **字典（dict）** 的形式，它记录了所有162条可搜索边上的操作，下面只给出了第一个block内前两个cell的结构的编码方式，可以看到key和value是对应边的名称和对应的one-hot操作：
```python
{
    'block0_cell0_node0_main': tensor([False, False,  True, False, False, False]),
    'block0_cell0_node0_skip0': tensor([False,  True]),
    'block0_cell0_node1_main': tensor([False, False,  True, False, False, False]),
    'block0_cell0_node1_skip0': tensor([False,  True]),
    'block0_cell0_node1_skip1': tensor([False,  True]),
    'block0_cell0_node2_main': tensor([ True, False, False, False, False, False]),
    'block0_cell0_node2_skip0': tensor([ True, False]),
    'block0_cell0_node2_skip1': tensor([False,  True]),
    'block0_cell0_node2_skip2': tensor([ True, False]),
    'block0_cell1_node0_main': tensor([False,  True, False, False, False, False]),
    'block0_cell1_node0_skip0': tensor([ True, False]),
    'block0_cell1_node1_main': tensor([False, False,  True, False, False, False]),
    'block0_cell1_node1_skip0': tensor([ True, False]),
    'block0_cell1_node1_skip1': tensor([False,  True]),
    'block0_cell1_node2_main': tensor([False, False, False, False,  True, False]),
    'block0_cell1_node2_skip0': tensor([False,  True]),
    'block0_cell1_node2_skip1': tensor([False,  True]),
    'block0_cell1_node2_skip2': tensor([False,  True]),
    ...
}
```

## 3.3 测试模型前向计算

In [3]:
B, F, T = 2, 80, 30
for T in [16, 32]:
    x = torch.rand(B, F, T)
    rm.reset() # 每次随机采样和激活一个新的子网
    y = model(x)
    print(y.shape)
    print(rm._cache, len(rm._cache))


torch.Size([2, 4, 49])
{'block0_cell0_node0_main': tensor([False, False,  True, False, False, False]), 'block0_cell0_node0_skip0': tensor([False,  True]), 'block0_cell0_node1_main': tensor([False, False, False, False,  True, False]), 'block0_cell0_node1_skip0': tensor([False,  True]), 'block0_cell0_node1_skip1': tensor([ True, False]), 'block0_cell0_node2_main': tensor([False, False, False, False, False,  True]), 'block0_cell0_node2_skip0': tensor([False,  True]), 'block0_cell0_node2_skip1': tensor([ True, False]), 'block0_cell0_node2_skip2': tensor([ True, False]), 'block0_cell1_node0_main': tensor([False,  True, False, False, False, False]), 'block0_cell1_node0_skip0': tensor([False,  True]), 'block0_cell1_node1_main': tensor([False, False, False,  True, False, False]), 'block0_cell1_node1_skip0': tensor([ True, False]), 'block0_cell1_node1_skip1': tensor([False,  True]), 'block0_cell1_node2_main': tensor([False, False,  True, False, False, False]), 'block0_cell1_node2_skip0': tensor

## 3.4 不同编码方式互相转换

我们首先设置了以列表格式定义的模型结构：
- 第一个节点：
    - main edge操作为linear
    - skip-connection edge的操作是索引值为1的操作，0表示zeroize，1表示identity
- ['conv5', 1, 0], # 第二个节点
- ['conv7d2', 1, 0, 1], # 第三个节点

In [4]:
list_desc = [
    ['linear', 1],
    ['conv5', 1, 0],
    ['conv7d2', 1, 0, 1],
]


通过`NASBenchASR`的 classmethod `list_desc_to_dict_mask`将其转换成字典格式，并赋值给`mask`变量

In [5]:
mask = NASBenchASR.list_desc_to_dict_mask(list_desc)
print(mask)

{'block0_cell0_node0_main': tensor([ True, False, False, False, False, False]), 'block0_cell0_node0_skip0': tensor([False,  True]), 'block0_cell0_node1_main': tensor([False,  True, False, False, False, False]), 'block0_cell0_node1_skip0': tensor([False,  True]), 'block0_cell0_node1_skip1': tensor([ True, False]), 'block0_cell0_node2_main': tensor([False, False, False, False,  True, False]), 'block0_cell0_node2_skip0': tensor([False,  True]), 'block0_cell0_node2_skip1': tensor([ True, False]), 'block0_cell0_node2_skip2': tensor([False,  True]), 'block0_cell1_node0_main': tensor([ True, False, False, False, False, False]), 'block0_cell1_node0_skip0': tensor([False,  True]), 'block0_cell1_node1_main': tensor([False,  True, False, False, False, False]), 'block0_cell1_node1_skip0': tensor([False,  True]), 'block0_cell1_node1_skip1': tensor([ True, False]), 'block0_cell1_node2_main': tensor([False, False, False, False,  True, False]), 'block0_cell1_node2_skip0': tensor([False,  True]), 'bloc

将`mask`作为参数传递给`NASBenchASR`初始化对应的子网结构。

注意，如果mask为None，会创建 Supernet；否则，会构建对应的子网结构，

In [6]:
model2 = NASBenchASR(mask=mask)
print(sum([p.numel() for p in model2.parameters()]))
y = model(x)


43102949


上面代码中，我们首先设置了以列表格式定义的模型结构，然后通过`NASBenchASR`的classmethod `list_desc_to_dict_mask`将其转换成字典格式

## 3.5 查询模型信息

In [7]:

print('All info:', model2.query_full_info(max_epochs=5))
print('FLOPs:', model2.query_flops())
print('Latency:', model2.query_latency())
print('#Params:', model2.query_params())
print('Test result:', model2.query_test_acc())
print('Val result:', model2.query_val_acc())

All info: {'val_per': [0.97343427, 0.8234044, 0.84591883, 0.85216177, 0.8556153], 'test_per': 0.8257796168327332, 'arch_vec': [(0, 1), (1, 1, 0), (4, 1, 0, 1)], 'model_hash': 'adb47992d93622245376905cc956a149', 'seed': 1234, 'jetson-nano-fp32': {'latency': 0.578345775604248}, 'gtx-1080ti-fp32': {'latency': 0.04792499542236328}}
FLOPs: 3845877266
Latency: {'jetson-nano-fp32': {'latency': 0.578345775604248}, 'gtx-1080ti-fp32': {'latency': 0.04792499542236328}}
#Params: 43100448
Test result: 0.24767844378948212
Val result: 0.22481237
